# Métodos Numericos 2023

## Parcial 2

### Fecha de entrega 

8 de Junio de 2023

### Nota sobre la programación en Julia

Recuerde que su notebook tiene que poder ser entendida por otra persona. Para ello:

* Cuando crea conveniente, use texto Markdown para desarrollar o comentar una idea.

* Agregue comentarios a su código de ser necesario.

* Realice gráficos que tengan etiquetas en los ejes y leyendas para cada curva o serie de puntos graficados, como así también un título apropiado.

* Procure respetar reglas de buena programación:

    * Use sangrías adecuadamente.

    * Cuando sea posible, implemente funciones de caracter general, en vez de código específico a cada problema, facilitando así la reutilización de código.

    * Implemente funciones que tengan los argumentos necesarios para su buen funcionamiento.

In [1]:
using Plots
using LaTeXStrings
using DataFrames
using CSV
using LsqFit

### Problema

**Contexto físico**

Los datos incluidos en el archivo adjunto `datos.dat` representan los tiempos de medición $t_i$ (primera columna) y posición $z_i$ (segunda columna) de una gota de agua en caída libre pero afectada por la resistencia impuesta por el aire. 
La posición se mide a lo largo de la perpendicular a la tierra ($z$), el origen de coordenadas se fija en el punto de partida de la gota, la cual tiene velocidad inicial nula.

Este experimento, ideado originalmente por el físico Philipp Lennard, ganador del Premio Nobel de Física en 1905 por sus investigaciones sobre los rayos catódicos y sus propiedades, consiste en medir durante la caída libre de la gota su posición para diferentes tiempos, los cuales, en este caso se escogieron equiespaciados.

A partir de suponer que la gota está sometida a la fuerza gravitacional que le ejerce la tierra y una fuerza restitutiva de rozamiento proporcional a su velocidad:
$$
\vec{F}=\vec{F_g} + \vec{F_R} = -mg \hat{z} - cv \hat{z}
$$
y usando las leyes de Newton, es posible deducir la posición $z$ en función del tiempo, como veremos más adelante en este parcial.
En este ejercicio no nos interesa que entiendan la física del experimento, pues lo verán más adeltante en la carrera, pero queremos que vean cómo se resuelven problemas físicos no triviales combinando variados métodos numéricos. 

**Incisos a resolver**

1. Lea los datos del archivo `datos.dat` y grafique las posiciones $z_i$ en función de los tiempos $t_i$. Los datos son tales que la unidad de tiempos es segundos $[\mathrm{s}]$ y la de posiciones es metros $[\mathrm{m}]$. Describa cualitativamente el comportamiento de la gota.

2. Calcule **numéricamente** la velocidad de la gota para cada uno de los tiempos de medición. Use el método centrado de cinco puntos en los puntos centrales, el método centrado de tres puntos en el segundo y el anteúltimo, y el de dos puntos en el primer y en el último punto. 

3. Grafique la aproximación de la velocidad $v_i$ calculada, en función del tiempo $t_i$. Describa cualitativamente el comportamiento temporal de la velocidad de la gota.

4. Grafique la velocidad $v_i$ en función de posición $z_i$. Describa cualitativamente cómo la velocidad de la gota cambia a medidad que cambia su posición.

5. Sabiendo que la ecuación que describe la posición de la gota en función del tiempo es:
    $$
    z(t)= - g\,\alpha\,t + g\,\alpha^2\, \left(1-e^{-t/\alpha} \right)
    $$
    donde $\alpha=m/c$, realice un ajuste de cuadrados mínimos de las posiciones $z_i$ en función de los tiempos $t_i$ y obtenga una estimación de los parámetros $\alpha$ y $g$. Verifique que los residuos son relativamente pequeños. Explicite los resultados del ajute en una celda en formato texto y LaTeX. En particular, compare el valor ajustado de $g$, con el comunmente utilizado de $g=9,81\,\mathrm{m}/\mathrm{s}^2$.

6. Use que $\alpha = m/c$ y que la masa de la gota es $m=0.001\,\mathrm{Kg}$ para determinar el valor de $c$. Explicite en una celda en formato texto y LaTex el valor obtenido de $c$. Además, especifique las unidades que tiene $\alpha$ y las uniades que tiene $c$.

7. Grafique la curva $z(t)$ ajustada con lo puntos especificados en el archivo `datos.dat` superpuestos.

8. Al descender la gota por efecto de la fuerza  gravitatoria que ejerce la tierra sobre ella al tiempo que es frenada por una fuerza restauradora proporcional a menos su velocidad (debida a la viscocidad del aire), la gota del experimento recibe un trabajo. La expresión de dicho trabajo, como debe usted recordar de sus clases de Física General I, está dada por:
    $$
    W = \int_{t_{\mathrm{inicial}}}^{t_{\mathrm{final}}} \vec{F}.\vec{v} \, dt    
    $$ 
    donde
    $$
    \vec{F} = -m\,g \hat{z} \; - c\,v(t)\hat{z}
    $$
    es la fuerza total actuante sobre la gota.

    Para calcular el trabajo, emplee el algoritmo de integración del método de Simpson compuesto para intervalos equiespaciados, utilizando los $t_i$ proveídos en los datos y las correspondientes aproximaciones de $v_i$ del inciso 2. Explicite el resultado del cómputo en una celda en formato texto y LaTeX.

    Aproxime la integral nuevamente, dividiendo el intervalo de tiempo en $100$ subintervalos ($n=101$ puntos) y usando correspondientes valores de $v(t)$ calculados a partir de la derivada analítica de la expresión del ajuste de $z(t)$. Compare resultados con la aproximación anterior.

9. Cuando la trayectoria del objeto no se cruza a si misma, una forma equivalente de calcular el trabajo proviene de la expresión:
    $$
    W = \int_{z_{\mathrm{inicial}}}^{z_{\mathrm{final}}} \; \vec{F}.\vec{dz} \nonumber \\
    $$
    Aproxime nuevamente el trabajo $W$ integrando esta otra expresión, utilizando el método del trapecio para intervalos **no equiespaciados**, y usando los puntos $z_i$ proveídos en los datos y las correspondientes aproximaciones $v_i$ del inciso 2.
    
    Compare resultados con el inciso anterior.
    
    **Ayuda:** Note que deberá implementar el el método del trapecio para intervalos **no equiespaciados**. Le recomendamos chequear el correcto funcionamiento del método, aproximando la integral
    
    $$
    \int_0^1 \sin(x)\,dx
    $$
    
    usando el siguiente arreglo de puntos de integración $x = [0.0,0.1,0.3,0.5,0.8,1.0]$. Debería darle $\approx 0.457421559686$.

10. Para verificar si las integrales le dieron el resultado correcto, calcule también el trabajo como:

    $$
    W=E_{\mathrm{cinética}}(t)-E_{\mathrm{cinética}}(0)
    $$
    
    donde
    
    $$
    E_{\mathrm{cinética}}(t) = \frac{1}{2}m v^2(t)
    $$

    Utilice para ello la velocidad final del ajuste. Imprima este valor también por pantalla.

11. A partir del archivo `datos.dat`, interpole la posición como función del tiempo con el método del polinomio interpolante de Lagrange dividiendo el intervalo de tiempo en $100$ subintervalos ($n=100$). Grafique, en un único gráfico, las siguientes curvas entre $t_i$ y $t_f$:
    1. Los puntos medidos $z_i$ vs $t_i$,
    2. el polinomio interpolante de Lagrange y
    3. la curva de la posición ajustada en el inciso 4.
   
12. Explique en una celda los resultados obtenidos.

### Respuestas

1. Lea los datos del archivo `datos.dat` y grafique las posiciones $z_i$ en función de los tiempos $t_i$. Los datos son tales que la unidad de tiempos es segundos $[\mathrm{s}]$ y la de posiciones es metros $[\mathrm{m}]$. Describa cualitativamente el comportamiento de la gota.

In [6]:
data = CSV.read("datos.dat", DataFrame, delim=" ", ignorerepeated=true)

t = data[!,1]
z = data[!,2]

17-element Vector{Float64}:
    0.0
   -4.412
  -15.971
  -32.69
  -53.134
  -76.3
 -101.397
 -127.914
 -155.443
 -183.709
 -212.492
 -241.716
 -271.065
 -301.263
 -331.608
 -358.733
 -394.825